# SageMaker SDK를 이용한 전체 학습 flow

## train, test, validate data 분리

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

df = pd.read_csv("kc_house_data.csv")

# 훈련, 검증 및 검정 분할
ys = np.array(df['price']).astype("float32")
xs = np.array(df['sqft_living']).astype("float32").reshape(-1,1)

np.random.seed(8675309)
train_features, test_features, train_labels, test_labels = train_test_split(xs, ys, test_size=0.2)
val_features, test_features, val_labels, test_labels = train_test_split(test_features, test_labels, test_size=0.5)

## Built-in algorithm을 이용한 학습

In [3]:
import sagemaker

# 모델 정의
linear_model = sagemaker.LinearLearner(role='arn:aws:iam::834160605896:role/service-role/AmazonSageMaker-ExecutionRole-20210317T125074',
                                       base_job_name='0521-flow-train',
                                       instance_count=1,
                                       instance_type='ml.m4.xlarge',
                                       predictor_type='regressor')

# 학습
train_records = linear_model.record_set(train_features, train_labels, channel='train')
val_records = linear_model.record_set(val_features, val_labels, channel='validation')
test_records = linear_model.record_set(test_features, test_labels, channel='test')

linear_model.fit([train_records, val_records, test_records], logs=False)

# 결과 확인
sagemaker.analytics.TrainingJobAnalytics(linear_model._current_job_name, metric_names = ['test:mse', 'test:absolute_loss']).dataframe()

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.



2021-05-21 03:30:47 Starting - Starting the training job
2021-05-21 03:30:49 Starting - Launching requested ML instances...............
2021-05-21 03:32:13 Starting - Preparing the instances for training..............
2021-05-21 03:33:27 Downloading - Downloading input data....
2021-05-21 03:33:52 Training - Downloading the training image....
2021-05-21 03:34:17 Training - Training image download completed. Training in progress..
2021-05-21 03:34:28 Uploading - Uploading generated training model.
2021-05-21 03:34:36 Completed - Training job completed


,timestamp,metric_name,value
0,0.0,test:mse,6.960262e+10
1,0.0,test:absolute_loss,1.754493e+05


## Hyperparameter Optimization

In [4]:
from sagemaker.parameter import (
    CategoricalParameter,
    ContinuousParameter,
    IntegerParameter,
    ParameterRange,
)
from sagemaker.amazon.hyperparameter import Hyperparameter
from sagemaker.tuner import HyperparameterTuner
import sagemaker

hyperparameter_ranges = {'learning_rate': ContinuousParameter(0.0001, 0.1, scaling_type='Logarithmic'),
                        'use_bias': CategoricalParameter([True, False])}

In [5]:
objective_metric_name = 'test:mse'
objective_type = 'Minimize'

In [18]:
tuner = HyperparameterTuner(linear_model,
                            objective_metric_name,
                            hyperparameter_ranges,
                            max_jobs=3,
                            max_parallel_jobs=3,
                            objective_type=objective_type,
                            base_tuning_job_name='0521-flow-hpo')

In [19]:
tuner.fit([train_records, val_records, test_records], include_cls_metadata=False)

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.


.......................................................!


In [21]:
# hyperparameter tuning 작업 결과 분석

import boto3

sagemaker_client = boto3.client('sagemaker')

job_name = tuner.latest_tuning_job.job_name

tuner_describe = sagemaker_client.describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=job_name)['HyperParameterTuningJobStatus']
tuner_describe

'Completed'

In [22]:
tuning_job_result = sagemaker_client.describe_hyper_parameter_tuning_job(HyperParameterTuningJobName=job_name)

tuning_job_result['BestTrainingJob']

{'TrainingJobName': '0521-flow-hpo-210521-0413-001-f79db8a4',
 'TrainingJobArn': 'arn:aws:sagemaker:ap-northeast-2:834160605896:training-job/0521-flow-hpo-210521-0413-001-f79db8a4',
 'CreationTime': datetime.datetime(2021, 5, 21, 4, 13, 50, tzinfo=tzlocal()),
 'TrainingStartTime': datetime.datetime(2021, 5, 21, 4, 16, 16, tzinfo=tzlocal()),
 'TrainingEndTime': datetime.datetime(2021, 5, 21, 4, 17, 27, tzinfo=tzlocal()),
 'TrainingJobStatus': 'Completed',
 'TunedHyperParameters': {'learning_rate': '0.06947714428339878',
  'use_bias': 'False'},
 'FinalHyperParameterTuningJobObjectiveMetric': {'MetricName': 'test:mse',
  'Value': 70205997056.0},
 'ObjectiveStatus': 'Succeeded'}

In [23]:
tuner_analysis = sagemaker.HyperparameterTuningJobAnalytics(job_name)
tuner_analysis.dataframe()

,learning_rate,use_bias,TrainingJobName,TrainingJobStatus,FinalObjectiveValue,TrainingStartTime,TrainingEndTime,TrainingElapsedTimeSeconds
0,0.010101,False,0521-flow-hpo-210521-0413-003-9feeecf7,Completed,7.078478e+10,2021-05-21 04:16:12+00:00,2021-05-21 04:17:46+00:00,94.0
1,0.002456,False,0521-flow-hpo-210521-0413-002-be17e10a,Completed,1.534885e+11,2021-05-21 04:16:26+00:00,2021-05-21 04:17:38+00:00,72.0
2,0.069477,False,0521-flow-hpo-210521-0413-001-f79db8a4,Completed,7.020600e+10,2021-05-21 04:16:16+00:00,2021-05-21 04:17:27+00:00,71.0


## Deploy Model

In [24]:
predictor = tuner.deploy(initial_instance_count=1, instance_type="ml.m4.xlarge")


2021-05-21 04:17:27 Starting - Preparing the instances for training
2021-05-21 04:17:27 Downloading - Downloading input data
2021-05-21 04:17:27 Training - Training image download completed. Training in progress.
2021-05-21 04:17:27 Uploading - Uploading generated training model
2021-05-21 04:17:27 Completed - Training job completed

Defaulting to the only supported framework/algorithm version: 1. Ignoring framework/algorithm version: 1.



-------------!

In [51]:
for i in range(0, 10):
    print(test_features[i])
    print(test_labels[i])

predictor.predict(test_features[0:10])

[1320.]
255000.0
[3020.]
808100.0
[1680.]
500000.0
[1300.]
676500.0
[1700.]
511718.0
[2740.]
625000.0
[3580.]
879900.0
[1770.]
355000.0
[7440.]
4208000.0
[1400.]
185000.0


[label {
   key: "score"
   value {
     float32_tensor {
       values: 344588.15625
     }
   }
 },
 label {
   key: "score"
   value {
     float32_tensor {
       values: 788375.9375
     }
   }
 },
 label {
   key: "score"
   value {
     float32_tensor {
       values: 438566.75
     }
   }
 },
 label {
   key: "score"
   value {
     float32_tensor {
       values: 339367.125
     }
   }
 },
 label {
   key: "score"
   value {
     float32_tensor {
       values: 443787.78125
     }
   }
 },
 label {
   key: "score"
   value {
     float32_tensor {
       values: 715281.5
     }
   }
 },
 label {
   key: "score"
   value {
     float32_tensor {
       values: 934564.875
     }
   }
 },
 label {
   key: "score"
   value {
     float32_tensor {
       values: 462061.40625
     }
   }
 },
 label {
   key: "score"
   value {
     float32_tensor {
       values: 1942224.125
     }
   }
 },
 label {
   key: "score"
   value {
     float32_tensor {
       values: 365472.28125
     }
  